# Import

In [1]:
import pandas as pd
import numpy as np

import copy
import matplotlib.pyplot as plt
import re
import seaborn as sns
import warnings

import nltk

from nltk.stem.snowball import SnowballStemmer 

sns.set()
#pd.set_option('max_columns', None)
warnings.filterwarnings('ignore')

# Task Description

I need to predict the price of a used cars based on its features. Obviously, this is a regression problem. As a control metric, I will choose Mean Absolute Percentage Error (MAPE). It is the mean of all absolute percentage errors between the predicted and actual values.
It both easy for end users to understand and simpler to compare model accuracy across use cases and datasets. 

Since the target is the price of a car in the hundreds of thousands of rubles, these values are far from 0, so I can use this metric without worrying about dividing by 0 when calculating the metric.

# Overview

In [2]:
data = pd.read_csv('data/data.csv')
data.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,Владение,ПТС,Привод,Руль,Состояние,Таможня,views,date_added,region,price
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,"['cruise-control', 'multi-wheel', 'xenon', 'he...",Автомобиль приобретался у официального дилера ...,1.8 LTR,180 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,...,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,76.0,24 сентября,в Тюмени,999000.0
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/110...,белый,"['cruise-control', 'multi-wheel', 'heated-wash...",Группа компаний Элан Моторс официальный дилер ...,1.6 LTR,110 N12,"{'cruise-control': True, 'glonass': True, 'asr...",бензин,...,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,259.0,24 сентября,в Санкт-Петербурге,1179000.0
2,лифтбек,SKODA,https://auto.ru/cars/new/group/skoda/rapid/217...,белый,"['cruise-control', 'heated-wash-system', 'airb...",Специальные предложения на автомобили в наличи...,1.6 LTR,90 N12,"{'cruise-control': True, 'glonass': True, 'esp...",бензин,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1464100.0
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,"['cruise-control', 'multi-wheel', 'heated-wash...",Купим Ваш автомобиль ДОРОГО Гарантированная с...,1.4 LTR,150 N12,"{'cruise-control': True, 'esp': True, 'usb': T...",бензин,...,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,31.0,25 сентября,в Тюмени,1420000.0
4,внедорожник 5 дв.,SKODA,https://auto.ru/cars/new/group/skoda/karoq/217...,серый,"['cruise-control', 'multi-wheel', 'heated-wash...",ЛОТ 01267595 Скидка на автомобиль при покупке ...,1.4 LTR,150 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",бензин,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2653190.0


In [3]:
data.shape

(130201, 36)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130201 entries, 0 to 130200
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              129986 non-null  object 
 1   brand                 129988 non-null  object 
 2   car_url               129969 non-null  object 
 3   color                 129988 non-null  object 
 4   complectation_dict    105267 non-null  object 
 5   description           129988 non-null  object 
 6   engineDisplacement    129988 non-null  object 
 7   enginePower           129988 non-null  object 
 8   equipment_dict        129969 non-null  object 
 9   fuelType              129986 non-null  object 
 10  image                 130065 non-null  object 
 11  mileage               103034 non-null  float64
 12  modelDate             129986 non-null  float64
 13  model_info            0 non-null       float64
 14  model_name            103034 non-null  object 
 15  

In [5]:
data.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня', 'views', 'date_added',
       'region', 'price'],
      dtype='object')

In [6]:
data.isnull().sum()

bodyType                   215
brand                      213
car_url                    232
color                      213
complectation_dict       24934
description                213
engineDisplacement         213
enginePower                213
equipment_dict             232
fuelType                   215
image                      136
mileage                  27167
modelDate                  215
model_info              130201
model_name               27167
name                        48
numberOfDoors              214
parsing_unixtime            47
priceCurrency               48
productionDate             213
sell_id                    232
super_gen                   66
vehicleConfiguration       214
vehicleTransmission        215
vendor                  130201
Владельцы                27167
Владение                115915
ПТС                      27167
Привод                   27169
Руль                     27168
Состояние                27169
Таможня                  27171
views   

In [7]:
data.describe()

,mileage,modelDate,model_info,numberOfDoors,parsing_unixtime,productionDate,sell_id,vendor,views,price
count,103034.000000,129986.000000,0.0,129987.000000,1.301540e+05,129988.000000,1.299690e+05,0.0,101998.000000,1.097650e+05
mean,116610.915407,2012.934578,NaN,4.643164,1.632842e+09,2015.083854,1.104866e+09,NaN,564.982284,2.801223e+06
std,85154.492046,6.017749,NaN,0.619354,1.316360e+05,5.929774,1.779786e+06,NaN,1814.036025,2.689068e+06
min,1.000000,1938.000000,NaN,2.000000,1.632613e+09,1952.000000,1.003139e+09,NaN,2.000000,2.300000e+04
25%,51000.000000,2010.000000,NaN,4.000000,1.632725e+09,2012.000000,1.104900e+09,NaN,127.000000,1.000000e+06
50%,100000.000000,2014.000000,NaN,5.000000,1.632821e+09,2016.000000,1.105219e+09,NaN,245.000000,2.045000e+06
75%,161000.000000,2017.000000,NaN,5.000000,1.632973e+09,2020.000000,1.105376e+09,NaN,511.000000,3.649000e+06
max,1000000.000000,2021.000000,NaN,5.000000,1.633049e+09,2021.000000,1.105403e+09,NaN,148149.000000,3.999000e+07


### Subtotal

- There are 130201 rows in this dataset.
- Target "price" contains 20436 missing values. It's bad, because I can't use this examples for model training, so I will remove them.
- Here a lot of categorical features, like as fuelType, brand, color and so on.
- There are description of announcement. It is text data and interesting, will I extract any valuable information from this feature.
- Here two features with links - car_url and image. I will not analyze image links because I will not work with CV techniques in this project. Also I will not use car_url, because I have already a good list of parameters for every case.
- complectation_dict - this feature contains useful information (for example, 'cruise-control', 'multi-wheel' and so on). Moreover, some parameters from this feature are duplicated in the main dataset. Next, I will analyze this feature in more detail.

# Data Preprocessing

In [8]:
# Deleting rows with missing values in target
data = data[data['price'].notna()].reset_index(drop=True)

In [9]:
# Let's remove (little bit later) model_info and vendor features, because they are empty
print(data['model_info'].isna().sum())
print(data['vendor'].isna().sum())

109765
109765


In [10]:
# Data has downloaded in one session. So let's check first value in parsing_unixtime column.
# Date of data parsing is 2021-09-26 
print(data['parsing_unixtime'][0])

1632613481.0


In [11]:
# Feature priceCurrency is constant. Let's remove it
print(data['priceCurrency'].value_counts())

RUB    109765
Name: priceCurrency, dtype: int64


In [12]:
# This feature is unnecessary, because dataset already has this content (body type, transmission, engine displacement). 
# Let's remove it
print(data['vehicleConfiguration'])

0                    LIFTBACK ROBOT 1.8
1                LIFTBACK AUTOMATIC 1.6
2               LIFTBACK MECHANICAL 1.6
3                    LIFTBACK ROBOT 1.4
4         ALLROAD_5_DOORS AUTOMATIC 1.4
                      ...              
109760             SEDAN MECHANICAL 2.0
109761     ALLROAD_5_DOORS VARIATOR 2.4
109762     ALLROAD_5_DOORS VARIATOR 2.0
109763     ALLROAD_5_DOORS VARIATOR 2.4
109764             SEDAN MECHANICAL 1.6
Name: vehicleConfiguration, Length: 109765, dtype: object


In [13]:
# Let's remove this feature, because missing values qty > 88%
print(data['Владение'].isna().sum())
print(f'Missing values qty: {round((data.Владение.isna().sum() / len(data)) * 100, 2)}%')

97597
Missing values qty: 88.91%


In [14]:
# Let's remove this feature, because it contains only one constant value and missing values
print(data['Состояние'].isna().sum())
print(data['Состояние'].value_counts())

21153
Не требует ремонта    88612
Name: Состояние, dtype: int64


In [15]:
# Let's remove this feature, because it contains only one constant value and missing values
print(data['Таможня'].isna().sum())
print(data['Таможня'].value_counts())

21156
Растаможен    88609
Name: Таможня, dtype: int64


In [16]:
# Deleting unnecessary features 
data = data.drop(columns=['car_url', 'image', 'model_info', 'vendor', 'parsing_unixtime', 'priceCurrency', 'vehicleConfiguration', 
                          'Владение', 'Таможня', 'Состояние'])

In [17]:
data.columns = ['body_type', 'brand', 'color', 'complectation_dict', 'description',
                'engine_displacement', 'engine_power', 'equipment_dict', 'fuel_type',
                'mileage', 'model_date', 'model_name', 'name', 'doors_number',
                'production_date', 'sell_id', 'super_gen', 'vehicle_transmission',
                'owners', 'vehicle_passport', 'wheel_drive', 'steering_wheel', 'views', 
                'date_added', 'region', 'price']

### body_type

In [18]:
counts_df = data['body_type'].value_counts().to_frame()
counts_df

,body_type
внедорожник 5 дв.,60847
седан,27685
лифтбек,6192
минивэн,3581
хэтчбек 5 дв.,3183
купе,2666
универсал 5 дв.,1542
компактвэн,1356
пикап двойная кабина,1248
хэтчбек 3 дв.,677


In [19]:
'''Let's replace unpopular body types in df (in comparison with other types) on 'other' type'''
unpopular_types = list(counts_df.loc[counts_df['body_type'] < 100].index)
data['body_type'] = ['other' if i in unpopular_types else i for i in data['body_type'].tolist()]

In [20]:
data['body_type'].value_counts()

внедорожник 5 дв.       60847
седан                   27685
лифтбек                  6192
минивэн                  3581
хэтчбек 5 дв.            3183
купе                     2666
универсал 5 дв.          1542
компактвэн               1356
пикап двойная кабина     1248
хэтчбек 3 дв.             677
фургон                    460
other                     327
Name: body_type, dtype: int64

###### VISUALIATION

In [21]:
'''Since the length of the dataset is > 109K rows, I can afford to delete one row'''
print('Missing values in body_type:', data['body_type'].isna().sum())
data = data.dropna(axis=0, subset=['body_type']).reset_index(drop=True)

Missing values in body_type: 1


### brand

In [22]:
data['brand'].value_counts().to_frame()

,brand
TOYOTA,16124
MERCEDES,14591
BMW,13743
NISSAN,13463
VOLKSWAGEN,12864
AUDI,9983
MITSUBISHI,8280
SKODA,8241
HONDA,4330
VOLVO,3400


###### VISUALIATION

### color

In [23]:
data['color'].value_counts().to_frame()

,color
чёрный,34600
белый,27212
серый,14656
синий,9717
серебристый,7966
красный,4786
коричневый,3801
бежевый,2443
зелёный,2026
голубой,1241


###### VISUALIATION

### fuel_type

In [24]:
counts_df = data['fuel_type'].value_counts().to_frame()
counts_df

,fuel_type
бензин,83931
дизель,24748
гибрид,985
электро,99
газ,1


In [25]:
'''Let's replace unpopular fuel types in df (in comparison with other types) on 'other' type'''
unpopular_types = list(counts_df.loc[counts_df['fuel_type'] < 1000].index)
data['fuel_type'] = ['other' if i in unpopular_types else i for i in data['fuel_type'].tolist()]

In [26]:
data['fuel_type'].value_counts()

бензин    83931
дизель    24748
other      1085
Name: fuel_type, dtype: int64

###### VISUALIATION

### model_name

In [27]:
data['model_name'].value_counts().to_frame()

,model_name
Tiguan,4162
X-Trail,3583
Touareg,2851
Camry,2767
Land Cruiser,2405
...,...
I,1
Succeed,1
Sprinter Carib,1
Carina ED,1


In [28]:
data['model_name'].isna().sum()

21152

In [76]:
'''Since model_name feature has 504 variations and 21152 missing values, let's drop it'''
data = data.drop(columns=['model_name'])

### name

In [156]:
data['name']

0                        Skoda Octavia III (A7)
1                                Skoda Rapid II
2                                Skoda Rapid II
3             Skoda Octavia III (A7) Рестайлинг
4                                 Skoda Karoq I
                          ...                  
109759                      Mitsubishi Lancer X
109760      Mitsubishi Outlander III Рестайлинг
109761    Mitsubishi Outlander III Рестайлинг 3
109762       Mitsubishi Outlander II Рестайлинг
109763           Mitsubishi Lancer X Рестайлинг
Name: name, Length: 109764, dtype: object

In [161]:
'''This feature contains information about brand, model and something else.
Let's try to extract car model from this feature. I parsed car models manually and keep them in brand_dict.
I'm fixing the car model if model from brand_dict contains in the feature name'''

brand_dict = {
    'skoda': ['Citigo', 'Rapid', 'Octavia', 'Fabia', 'Superb', 'Kodiaq', 'Karoq', 'Scala', 'Kamiq', 'Enyaq', 'Kushaq', 'Slavia', 'Yeti', 'Felicia',
             'Roomster', 'Favorit', '105'],
    
    'audi': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'Cabriolet', 'TT', 'Coupe', 'Roadster', 'Q2', 'Q3', 'Q5', 'Q7', 'R8', 'e-tron', 'R8', 
             'Sportback', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'SQ2', 'SQ5', 'SQ7', 'TTS', '100', '200', '80', '90', 'e', 'q8', 'rs', 'sq8', 'v8'],
    
    'honda': ['Accord', 'Avancier', 'City', 'Civic', 'Concerto', 'CR-V', 'CR-X', 'CR-Z', 'Crossroad', 'Crosstour', 'Element', 'Elevate', 'Elysion', 
              'Clarity', 'Fit', 'FR-V', 'Freed', 'Grace', 'HR-V', 'Integra', 'S660', 'Vigor', 'Insight', 'Insight', 'Jazz', 'Legend', 'Logo', 'Slash', 
              'N-ONE', 'N-WGN', 'NSX', 'Odyssey', 'Orthia', 'Pilot', 'Prelude', 'Rafaga', 'Ridgeline', 'S-MX', 'S2000', 'Shirui', 'Shuttle', 'Stream',
              'Torneo', 'Zest', 'ZR-V', 'stepwgn', 'acty', 'airwave', 'ascot', 'capa', 'cr', 'domani', 'edix', 'fr', 'hr', 'inspire', 'jade', 'life', 
              'mobilio', 'n', 'partner', 's', 'saber', 'stepwgn', 'today', 'vamos', 'vezel', 'z'],
    
    'volvo': ['66', '140', '200', '300', '460', '480', '700', '850', '900', '940', '960', 'Amazon', 'Brage', 'C30', 'C70', 'C202', 'EX30', 'EX90', 'F4', 
              'F10', 'F85', 'F88', 'FE', 'FH', 'FL', 'FM', 'FMX', 'L340', 'Longnose', 'LV4', 'LV60', 'LV66', 'LV71', 'LV76', 'LV81', 'LV120', 'PV4', 
              'P1800', 'P1900', 'Carioca', 'PV444', 'S40', 'S60', 'S70', 'S80', 'S90', 'Sharpnose', 'Snabbe', 'T6', 'Titan', 'V60', 'Viking', 'VM', 'VN', 
              'VHD', 'VT', 'XC40', 'XC60', 'V70', 'XC90', '240', '440', '740', '760', 'v40', 'v50', 'v90', 'xc70'],
    
    'bmw': ['328', '501', '503', '507', '700', '3200', 'ActiveHybrid', 'E3', 'E9', 'E12', 'E21', 'E23', 'E24', 'E28', 'E30', 'E31', 'E32', 'E34', 'E34', 
            'E36', 'E38', 'E39', 'E46', 'E60', 'E65', 'E87', 'E90', 'E92', 'F01', 'F10', 'F22', 'F23', 'F26', 'F30', 'F34', 'F82', 'G02', 'G11', 'G15', 
            'G20', 'G30', 'G60', 'G70', 'G82', 'GINA', 'Hydrogen', 'Vision', 'i3', 'i7', 'i8', 'Isetta', 'iX', 'M1', 'M3', 'M5', 'M6', 'Klasse', 'X', 
            'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'XM', 'Z', 'Z1', 'Z4', 'Z8', 'm2', 'm8', 'z3', '320d', 'gran', '330i', '645i', 'f87', '318i', 
            '640d', '640i','116d', '116i', '118i', '120d', '135i', '316d', '316i', '318d', '320i', '328i', '330d', '335i', '420d', '420i', '428i', 
            '430i', '435i', '630i', '650i', 'active', 'm340i'],
    
    'nissan': ['180SX', '300ZX', '350Z', '370Z', 'Almera', 'Hybrid', 'Armada', 'Atlas', 'Atleon', 'Avenir', 'Thumb', 'Bluebird', 'Sylphy', 'C80', 'Caball', 
               'Caravan', 'Elgrand', 'Cedric', 'Cefiro', 'Condor', 'Cube', 'Cherry', 'DeltaWing', 'Z', 'Figaro', 'Fuga', 'Gloria', 'GT-R', 'Nismo', 'Truck', 
               'II', 'Juke', 'Kicks', 'Laurel', 'Spirit', 'LEAF', 'Leopard', 'Liberty', 'Livina', 'Logan', 'Magnite', 'Maxima', 'Micra', 'Mixim', 'Murano', 
               'Navara', 'NP300', 'NT400', 'NT500', 'NV200', 'NV250', 'NV400', 'III', 'Pao', 'Pathfinder', 'Patrol', 'Pintara', 'Pivo', 'A-Star', 'Presage', 
               'President', 'Primera', 'Homer', 'Royal', 'Pulsar', 'GTI-R', 'Qashqai', 'Quest', 'Quon', 'R’nessa', 'Rasheen', 'Project', 'S30', 'Sakura', 
               'Sentra', 'Serena', 'Silvia', 'Skyline', 'Stagea', 'Stanza', 'Sunny', 'Wagon', 'Teana', 'Terrano', 'Tiida', 'Titan', 'Townstar', 'Trade', 
               'Truck', 'Vanette', 'Wingroad', 'X-Trail', 'Xterra', 'bassara', 'dayz', 'dualis', 'expert', 'gt', 'kix', 'march', 'note', 'nv100', 
               'qashqai2', 'rnessa', 'tino', 'x'],
    
    'infiniti': ['Q50', 'Q60', 'Q70', 'QX30', 'QX50 ', 'QX60', 'QX80', 'M30', 'M45', 'G20', 'J30', 'QX4', 'I', 'Q45', 'FX35', 'QX56', 'QX70', 'G25', 
                 'G35', 'G37', 'fx', 'jx', 'm', 'q30', 'qx50', 'qx55'],
    
    'mercedes': ['300SL', '500K', '770', 'R129', 'A-класс', 'GT', 'Atron', 'B-класс', 'Patent-Motorwagen', 'C-класс', 'C111', 'C215', 'C218', 
                 'C219', 'C112', 'C117', 'C257', 'Citan', 'CL-класс', 'CLA-класс', 'CLC-класс', 'GTR', 'CLS-класс', 'E-класс', 'SUV', 'F-Cell', 'G-класс', 
                 'G4', 'GLB-класс', 'GLE-класс', 'GLK-класс', 'GLS-класс', 'MB100', 'D', 'SLR', '6.3', '6.9', 'AMG', 'V167', 'X167', 'L206', 'E7.4RS', 
                 'TN', 'Vaneo', 'W108', 'W111', 'W112', 'W180', 'R171', 'R-класс', 'ONE', 'R107', 'C107', 'R170', 'R172', 'R230', 'R231', 'S', 'S-класс', 
                 'McLaren', 'SL-класс', 'AMG', 'SLC-класс', 'SLK-класс', 'AMG', 'Sprinter', 'T80', 'Т-класс', 'Transporter', '300', 
                 'V-класс', 'W21', 'W212', 'CLK-класс', 'W100', 'W109', 'W110', 'W113', 'W114', 'W115', 'W116', '190SL', 'W120', 'W123', 'W124', 
                 'Rekordwagen', 'W126', 'W136', 'W140', 'W143', 'W163', 'W164', 'W166', 'W177', 'W194', 'W198', 'W201', 'W202', 'W203', 'W204', 
                 'W205', 'W208', 'W209', 'W210', 'W211', 'W213', 'W214', 'W220', 'W221', 'W222', 'W223', 'W245', 'W246', 'W251', 'W460', 'W463', 
                 'Vito', 'W901', 'W906', 'W907', 'X-класс', 'GLA-класс', 'X164', 'X166'],
    
    'toyota': ['4Runner', '2000GT', 'AA', 'AE86', 'Agya', 'Allion', 'Alphard', 'Aqua', 'Aurion', 'Auris', 'Avalon', 'Avensis', 'Aygo', 'X', 'bB', 
               'Belta', 'Brevis', 'bZ3', 'Caldina', 'Camry', 'Solara', 'Carina', 'ED', 'Celica', 'Camry', 'GT-Four', 'Century', 'CH-R', 'Chaser', 'Corolla', 
               'E140', 'E170', 'Cross', 'Spacio', 'Corona', 'Cressida', 'Cresta', 'Crown', 'Majesta', 'Curren', 'Cynos', 'Duet', 'Dyna', 'eCom', 'Fine-N', 
               'Fine-X', 'Cruiser', 'Fortuner', 'FT-HS', 'Gaia', 'Highlander', 'Harrier', 'Hiace', 'Highlander', 'Hilux', 'X', 'Ipsum', 'iQ', 'Isis', 'iX4', 
               'Cruiser', 'Prado', 'LiteAce', 'II', 'Qualis', 'X', 'Matrix', 'Mirai', 'MR2', 'Nadia', 'Noah', 'Opa', 'Passo', 'Picnic', 'Epoch', 'Space', 'PM', 
               'Porte', 'Premio', 'Previa', 'Prius','PHV', 'α', 'ProAce', 'Probox', 'Progres', 'Publica', 'Ractis', 'Raum', 'RAV4', 'EV', 'Rush', 'SA', 
               'SAI', 'Sequoia', 'Sienna', 'Soarer', '800', 'Sprinter', 'Marino', 'Stout', 'Succeed', 'Supra', 'Tacoma', 'Tercel', 'Ace', 'Toyoace', 
               'Avanza', 'Hybrid', 'Cross', 'Hybrid', 'Tundra', 'Cruiser', 'Venza', 'Verossa', 'Vista', 'V20', 'Wish', 'Yaris', 'allex', 'c', 
               'cavalier', 'esquire', 'estima', 'funcargo', 'platz', 'regius', 'sienta', 'spade', 'starlet', 'verso', 'vitz', 'voxy', 'windom'],
    
    'lexus': ['ES', 'LS', 'LX', 'GX', 'NX', 'RX', 'UX', 'ct', 'gs', 'hs', 'is', 'lc', 'rc', 'sc'], 

    'volkswagen': ['Golf', 'CrossFox', 'Eos', 'GLI', 'Gol', 'Golf', 'GTI', 'Jetta', 'Wagon', 'Beetle', 'Passat', 'CC', 'Routan', 'SportVan', 'Tiguan', 
                   'Touareg', 'Caddy', 'Fox', 'Gol', 'GTI', 'Parati', 'Polo', 'Saveiro','amarok', 'arteon', 'bora', 'california', 'caravelle', 
                   'multivan', 'phaeton', 'pointer', 'scirocco', 'sharan', 'taos', 'teramont', 'touran', 'transporter', 'type'],
    
    'mitsubishi': ['ASX', 'Attrage', 'Carisma', 'Delica', 'Diamante', 'Dingo', 'Dion', 'E-More', 'Eclipse', 'Endeavor', 'Eupheme', 'EV', 'Galant', 'Lambda', 
                   'Jingang', 'Jeep', 'L200', 'Lancer', 'Evolution', 'Minica', 'Mirage', '500', 'Cross', 'Canter', 'Fighter', 'Rosa', 'Great', 'HSR', 'Magna', 
                   'Minicab', 'Raider', 'Outlander', 'Pajero', 'Mini', 'Sport', 'Runner', 'Star', 'Starion', 'Box', 'Tredia', '73', 'Harrier', 'Hiace', 
                   'Highlander', 'Hilux', 'X', 'Ipsum', 'iQ', 'Isis', 'iX4', 'Cruiser', 'Prado', 'LiteAce', 'II', 'Qualis', 'X', 'Matrix', 'Mirai', 'MR2', 'Nadia',  
                   'Opa', 'Passo', 'Picnic', 'Epoch', 'Space', 'PM', 'Porte', 'Premio', 'Previa', 'Prius', 'PHV', 'ProAce', 'Probox', 'Progres', 'Publica', 
                   'Ractis', 'Raum', 'RAV4', 'EV', 'Rush', 'SA', 'SAI', 'Sequoia', 'Sienna', 'Soarer', '800', 'Sprinter', 'Marino', 'Stout', 'Succeed', 'Supra', 
                   'Tacoma', 'Tercel', 'Ace', 'Toyoace', 'Avanza', 'Hybrid', 'Cross', 'Hybrid', 'Tundra', 'Cruiser', 'Venza', 'Verossa', 'Vista', 'V20', 
                   'Wish', 'Yaris', 'Noah', '3000', 'airtrek', 'challenger', 'chariot', 'colt', 'ek', 'eterna', 'fto', 'grandis', 'i', 'l400', 'legnum', 
                   'libero', 'montero', 'rvr', 'toppo']
}

In [162]:
norm_name_list = [norm_string(s).split(' ') for s in data['name'].tolist()]
model_list = []
none_list = []

for s, norm_name in enumerate(norm_name_list):
    brand = norm_string(data['brand'][s])
    brand_model_list = brand_dict.get(brand)

    model = [a for a in brand_model_list if norm_string(a) in norm_name]
    
    if model:
        model_list.append(model[0])

    else:
        model_list.append(None)
        none_list.append(norm_name)

In [163]:
len(none_list)

1451

In [164]:
none_list

[['mercedes', 'benz', 'gle', 'coupe', '400', 'i', 'c292'],
 ['mercedes', 'benz', 'a', 'класс', '180', 'ii', 'w169', 'рестайлинг'],
 ['mercedes', 'benz', 'maybach', 'gls', '600', 'i'],
 ['mercedes', 'benz', 'gle', 'coupe', '400', 'i', 'c292'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'maybach', 'gls', '600', 'i'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'gla', '250', 'i', 'x156'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'gla', '250', 'i', 'x156'],
 ['mercedes', 'benz', 'a', 'класс', '160', 'i', 'w168', 'рестайлинг'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'glc', '200', 'i', 'x253', 'рестайлинг'],
 ['mercedes', 'benz', 'gla', '250', 'i', 'x156', 'рестайлинг'],
 ['mercedes', 'benz',

In [ ]:
data['model'] = model_list

In [160]:
data['model'].value_counts().to_frame().head(60)

,model
Benz,14545
III,5857
Tiguan,4710
Cruiser,3948
II,3833
A6,3673
Camry,3373
Outlander,3217
Touareg,2876
X5,2726


# Help Functions

In [33]:
def norm_string(string: str, stemming_flag = 0):
    stemmer = SnowballStemmer("russian") 

    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~«»»"–+»+"-"”№'''
    string = re.sub('\,|\.|\/|\-', ' ', string)
    string = re.sub('\s+', ' ', string)
    string = [i for i in string if i not in punc] # punc delete
    string = ''.join(string).lower()

    if stemming_flag == 1:
        string = nltk.word_tokenize(string)
        string = [stemmer.stem(w) if len(w) > 6 else w for w in string]
        string = ' '.join(string).lower()

    return string